In [1]:
import random
import time
import math
import csv
import os
import chessAI
from pathlib import Path
from time import perf_counter, monotonic
from multiprocessing import Process, Queue
from chessEngine import GameState, Move
from chessAI import get_best_move, run_ai_loop
from stockfishHandler import StockfishPlayer

/Users/arjunvijayanup/Documents/3rd Trimester/Project in Maths Modelling/Chess/project-chess-final/project-ai-chess-engine/chess_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Deterministic check for move search

In [2]:
def best_move_once():
    gs = GameState()
    legal = gs.get_valid_moves()
    random.seed(22)  # stabilize any random tie-breakers
    mv = get_best_move(gs, legal)
    return str(mv)

In [3]:
a = best_move_once()
b = best_move_once()
print("Run1:", a)
print("Run2:", b)
print("DETERMINISTIC" if a == b else "NON-DETERMINISTIC")

Run1: d4
Run2: d4
DETERMINISTIC


# Move generation Correctness & Speed

In [4]:
def perft(gs, depth):
    if depth == 0:
        return 1
    nodes = 0
    for m in gs.get_valid_moves():
        gs.make_move(m)
        nodes += perft(gs, depth - 1)
        gs.undo_move()
    return nodes

In [5]:
def divide(gs, depth):
    total = 0
    for m in gs.get_valid_moves():
        gs.make_move(m)
        c = perft(gs, depth - 1)
        gs.undo_move()
        name = getattr(m, "get_chess_notation", lambda: str(m))()
        print(f"{name:<8} {c}")
        total += c
    print(f"Total: {total}")
    return total

In [6]:
gs = GameState()
tests = [(1, 20), (2, 400), (3, 8902), (4, 197281)]
for d, expected in tests:
    t0 = time.time()
    n = perft(gs, d)
    dt = time.time() - t0
    status = "PASS" if n == expected else f"FAIL (expected {expected})"
    print(f"depth {d}: nodes={n}  {status}  time={dt:.3f}s  nps={int(n/max(dt,1e-6))}")

depth 1: nodes=20  PASS  time=0.003s  nps=7006
depth 2: nodes=400  PASS  time=0.049s  nps=8245
depth 3: nodes=8902  PASS  time=0.994s  nps=8956
depth 4: nodes=197281  PASS  time=22.564s  nps=8743


# Stockfish vs Our AI engine (ELO & Match Stats)

-   N = wins + draws + losses.
-   Score S = (wins + 0.5*draws) / N.
-   Rating difference $\Delta$ vs that Stockfish config (depth/time) using the Elo logistic model:

$$
\Delta = -400 \log_{10}\!\left(\frac{1}{S} - 1\right)
$$

In [7]:
def elo_diff_from_score(score):
    score = min(max(score, 1e-6), 1-1e-6)
    return -400.0 * math.log10(1/score - 1)

In [19]:
NUM_GAMES = 50
OUR_DEPTH = 4
STOCKFISH_DEPTH = 4
MAX_PLIES = 300
AI_TIMEOUT_SEC = 90
PRINT_PER_MOVE = False

chessAI.MAX_DEPTH = OUR_DEPTH

LOG_TO_CSV = True

In [9]:
def _ensure_csv_header(path):
    need_header = not os.path.exists(path) or os.path.getsize(path) == 0
    if need_header:
        with open(path, "w", newline="") as f:
            w = csv.writer(f)
            w.writerow([
                "game_index", "ai_color", "result",
                "plies", "sf_depth", "ai_depth",
                "ai_nodes", "ai_time_ms", "ai_nps",
                "tt_probes", "tt_hits", "tt_hit_rate",
                "tt_stores", "beta_cutoffs", "first_move_cutoffs",
                "avg_branch", "killer_uses", "history_uses",
                "ai_moves", "sf_time_ms", "game_wall_ms",
                "did_castle", "did_en_passant", "did_promotion"
            ])

def _append_csv_row(path, row):
    with open(path, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow(row)


In [10]:
def _empty_agg():
    return {
        "nodes": 0,
        "ms": 0.0,
        "sf_ms": 0.0,
        "tt_probes": 0,
        "tt_hits": 0,
        "tt_stores": 0,
        "beta_cutoffs": 0,
        "first_move_cutoffs": 0,
        "killer_uses": 0,
        "history_uses": 0,
        "branch_sum": 0,
        "branch_samples": 0,
        "plies": 0,   # total half-moves in the game
        "ai_moves": 0 # number of AI root searches performed
    }


def _accumulate(agg, stats):
    """Add a single SearchStats into the aggregate dict."""
    agg["nodes"] += getattr(stats, "nodes")
    agg["ms"] += getattr(stats, "ms")
    agg["tt_probes"] += getattr(stats, "tt_probes")
    agg["tt_hits"] += getattr(stats, "tt_hits")
    agg["tt_stores"] += getattr(stats, "tt_stores")
    agg["beta_cutoffs"] += getattr(stats, "beta_cutoffs")
    agg["first_move_cutoffs"] += getattr(stats, "first_move_cutoffs")
    agg["killer_uses"] += getattr(stats, "killer_uses")
    agg["history_uses"] += getattr(stats, "history_uses")
    agg["branch_sum"] += getattr(stats, "branch_sum")
    agg["branch_samples"] += getattr(stats, "branch_samples")
    agg["ai_moves"] += 1


def _format_agg(agg):
    """Human-readable one-liner for game stats."""
    nodes = agg["nodes"]
    ms = agg.get("ms", 0.0)
    sf_ms = agg.get("sf_ms", 0.0)
    nps = (nodes / (ms / 1000.0)) if ms > 1e-9 else 0.0
    ttp = agg["tt_probes"]
    tth = agg["tt_hits"]
    tt_hit_rate = (100.0 * tth / ttp) if ttp else 0.0
    avg_branch = (agg["branch_sum"] / agg["branch_samples"]) if agg["branch_samples"] else 0.0
    return (
        f"nodes={nodes:,}, time={ms:.3f} ms, NPS={nps:,.0f}, "
        f"TT hit={tt_hit_rate:.1f}% ({tth}/{ttp}), stores={agg['tt_stores']}, "
        f"beta-cutoffs={agg['beta_cutoffs']}, first-move-cutoffs={agg['first_move_cutoffs']}, "
        f"avg-branch={avg_branch:.2f}, killer-uses={agg['killer_uses']}, "
        f"history-uses={agg['history_uses']}, AI-moves={agg['ai_moves']}, plies={agg['plies']}, "
        f"SFtime={sf_ms:.3f} ms"
    )

def _specials_summary(flags):
    """
    flags values: {"castling": true/false, "en_passant": true/falsel, "promotion": true/false}
    """
    tags = []
    if flags.get("castling"):     tags.append("castling")
    if flags.get("en_passant"):   tags.append("en passant")
    if flags.get("promotion"):    tags.append("promotion")
    return ", ".join(tags) if tags else "none"


In [11]:
def start_ai_worker(depth):
    iq, oq = Queue(), Queue()
    print(f"[child] chessAI.MAX_DEPTH set to {depth}")
    p = Process(target=chessAI.run_ai_loop, args=(iq, oq, depth), daemon=True)
    p.start()
    return p, iq, oq


def stop_ai_worker(proc):
    if proc and proc.is_alive():
        proc.terminate()
        proc.join(timeout=2)

In [12]:
def get_ai_move(worker_in, worker_out, gs, legal_moves):
    pos_key = len(gs.moves_log)
    worker_in.put((pos_key, gs, legal_moves))
    deadline = perf_counter() + AI_TIMEOUT_SEC
    while perf_counter() < deadline:
        if not worker_out.empty():
            pos_key_ret, mv, stats = worker_out.get()
            if pos_key_ret == len(gs.moves_log):
                return mv, stats
    return None, None # timeout

In [13]:
def play_one_game(sf, worker_in, worker_out, ai_is_white=True):
    gs = GameState()
    legal = gs.get_valid_moves()
    plies = 0
    agg = _empty_agg()
    moves_text = []  # SAN/your engine’s string form
    specials = {"castling": False, "en_passant": False, "promotion": False}

    while True:
        # terminal?
        if not legal: # no legal moves left
            if gs.is_stalemate:
                agg["plies"] = plies
                return "Stalemate", agg, moves_text, specials, gs
            winner_is_white = not gs.white_to_move  # side NOT to move was just checkmated
            agg["plies"] = plies
            if winner_is_white == ai_is_white:
                return "ai", agg, moves_text, specials, gs
            else:
                return "stockfish", agg, moves_text, specials, gs
            #return "ai", agg if winner_is_white == ai_is_white else "stockfish", agg

        if getattr(gs, "is_threefold_repetition", False) or getattr(gs, "is_fifty_move_draw", False) or getattr(gs, "is_insufficient_material", False):
            agg["plies"] = plies
            if getattr(gs, "is_threefold_repetition", False):
                return "Draw (Three-Fold Repetetion)", agg, moves_text, specials, gs
            if getattr(gs, "is_fifty_move_draw", False):
                return "Draw (Fifty Move Draw)", agg, moves_text, specials, gs
            if getattr(gs, "is_insufficient_material", False):
                return "Draw (Insufficient Material)", agg, moves_text, specials, gs
            else:
                return "Draw (Rule-Based)", agg, moves_text, specials, gs
        
        if plies >= MAX_PLIES:
            agg["plies"] = plies
            return "Draw (Plies exceeded 300)", agg, moves_text, specials, gs

        ai_turn = (gs.white_to_move and ai_is_white) or ((not gs.white_to_move) and (not ai_is_white))
        move_source = "ai" if ai_turn else "stockfish"
        mv = None

        if ai_turn:
            t_ai0 = perf_counter()
            mv, stats = get_ai_move(worker_in, worker_out, gs, legal)
            agg["ms"] = agg.get("ms", 0.0) + (perf_counter() - t_ai0) * 1000.0
            
            if mv is None:  # timeout / fail-safe → resign
                agg["plies"] = plies
                return "stockfish", agg, moves_text, specials
            _accumulate(agg, stats)
            if PRINT_PER_MOVE:
                print("AI move statistics:", _format_agg({**agg, "plies": plies}) )
        else:
            t_sf0 = perf_counter()
            mv = sf.get_best_move(gs)
            agg["sf_ms"] = agg.get("sf_ms", 0.0) + (perf_counter() - t_sf0) * 1000.0

        # Special move flags
        if getattr(mv, "is_castling_move", False):
            specials["castling"] = True
        if getattr(mv, "is_en_passant", False):
            specials["en_passant"] = True
        # Pawn promotion
        if getattr(mv, "is_pawn_promotion", False) or getattr(mv, "promotion_choice", None):
            specials["promotion"] = True

        # make move (promotion default 'Q')
        gs.make_move(mv)
        moves_text.append(str(mv))
        legal = gs.get_valid_moves()
        plies += 1


In [ ]:
grand_ai_wins = grand_sf_wins = grand_draws = 0

try:
    for aid in range(3, OUR_DEPTH+1):
        for sfd in range(3, STOCKFISH_DEPTH+1):
            if sfd<=aid:
                ai_wins = sf_wins = draws = 0
                total_ai_ms = 0.0
                total_ai_moves = 0
                sf = StockfishPlayer(base_dir=os.getcwd(), depth=sfd)
                worker_proc, worker_in, worker_out = start_ai_worker(aid)
                CSV_PATH = f"arena_results_sf{sfd}_ai{aid}.csv"
                _ensure_csv_header(CSV_PATH)
                print("\n=======++++++=======")
                print(f"AI_depth:       {aid}")
                print(f"Stockfish_depth:{sfd}")
                print("\n=======++++++=======")
                for g in range(NUM_GAMES):
                    ai_is_white = (g % 2 == 0)
                    t0 = perf_counter()
                    result, agg, moves_text, specials, gs = play_one_game(sf, worker_in, worker_out, ai_is_white=ai_is_white)
                    
                    if result == "ai":
                        ai_wins += 1
                    elif result == "stockfish":
                        sf_wins += 1
                    else:
                        draws += 1
                    
                    print(f"Game {g+1}/{NUM_GAMES}: {result} ({'AI as White' if ai_is_white else 'AI as Black'})")
                    print("Stats:", _format_agg(agg))
                    # special move summary
                    print("Special moves:", _specials_summary(specials))

                    nodes = agg["nodes"]
                    ms = agg["ms"]
                    nps = (nodes / (ms / 1000.0)) if ms > 1e-9 else 0.0
                    ttp = agg["tt_probes"]
                    tth = agg["tt_hits"]
                    tt_hit_rate = (100.0 * tth / ttp) if ttp else 0.0
                    avg_branch = (agg["branch_sum"] / agg["branch_samples"]) if agg["branch_samples"] else 0.0
                    game_wall_ms = (perf_counter() - t0) * 1000.0 
                    sf_ms = agg.get("sf_ms", 0.0)


                    _append_csv_row(CSV_PATH, [
                            g + 1,
                            "white" if ai_is_white else "black",
                            result,
                            agg["plies"],
                            sfd,
                            aid,
                            nodes,
                            f"{ms:.1f}",
                            f"{nps:.0f}",
                            ttp,
                            tth,
                            f"{tt_hit_rate:.1f}",
                            agg["tt_stores"],
                            agg["beta_cutoffs"],
                            agg["first_move_cutoffs"],
                            f"{avg_branch:.2f}",
                            agg["killer_uses"],
                            agg["history_uses"],
                            agg["ai_moves"],
                            f"{sf_ms:.1f}",
                            f"{game_wall_ms:.1f}",
                            bool(specials.get("castling")),
                            bool(specials.get("en_passant")),
                            bool(specials.get("promotion"))
                        ])

                total = ai_wins + sf_wins + draws
                print("\n=== Final Stats ===")
                print(f"AI wins:        {ai_wins}")
                print(f"Stockfish wins: {sf_wins}")
                print(f"Draws:          {draws}")
                if total:
                    score = (ai_wins + 0.5*draws) / total
                    print(f"AI score: {100*score:.1f}%")    
                    print(f"Elo vs SF@depth{STOCKFISH_DEPTH}: {elo_diff_from_score(score):+.0f}")
                grand_ai_wins += ai_wins
                grand_sf_wins += sf_wins
                grand_draws += draws

    if LOG_TO_CSV:
        print(f"\nCSV saved to: {os.path.abspath(CSV_PATH)}")
finally:
    stop_ai_worker(worker_proc)
    sf.quit_engine()

overall = grand_ai_wins + grand_sf_wins + grand_draws
if overall:
    print("\n=== Overall across all AI and Stockfish search depth pairs ===")
    print(f"AI wins:        {grand_ai_wins}")
    print(f"Stockfish wins: {grand_sf_wins}")
    print(f"Draws:          {grand_draws}")

[child] chessAI.MAX_DEPTH set to 4

=======++++++=======
AI_depth:       4
Stockfish_depth:4

=======++++++=======
Game 1/50: Draw (Three-Fold Repetetion) (AI as White)
Stats: nodes=95,746, time=548338.105 ms, NPS=175, TT hit=13.9% (13297/95746), stores=82896, beta-cutoffs=81474, first-move-cutoffs=52042, avg-branch=32.00, killer-uses=12079, history-uses=12828, AI-moves=89, plies=178, SFtime=90.386 ms
Special moves: castling, promotion
Game 2/50: Draw (Three-Fold Repetetion) (AI as Black)
Stats: nodes=85,416, time=544348.160 ms, NPS=157, TT hit=11.5% (9812/85416), stores=75788, beta-cutoffs=76820, first-move-cutoffs=57374, avg-branch=34.02, killer-uses=12442, history-uses=12952, AI-moves=45, plies=91, SFtime=96.181 ms
Special moves: castling
Game 3/50: Draw (Three-Fold Repetetion) (AI as White)
Stats: nodes=73,903, time=518990.013 ms, NPS=142, TT hit=13.5% (10002/73903), stores=64046, beta-cutoffs=65203, first-move-cutoffs=48985, avg-branch=38.25, killer-uses=3135, history-uses=3265, A